In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

In [3]:
import torch
import config
import pytorch_lightning as pl
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule
from classification.models.CRNN import CRNNPLModule

In [4]:
from pytorch_lightning.callbacks import Callback

class SaveCallback(Callback): 
    def __init__(self, model_name):
        super().__init__()
        self.model_name = model_name
        self.best_val_acc = None

    def on_epoch_end(self, trainer, pl_module):
        if not self.best_val_acc or pl_module.val_results_history[-1]["val_acc"] > self.best_val_acc:
            print("new best val acc", pl_module.val_results_history[-1]["val_acc"])
            self.best_val_acc = pl_module.val_results_history[-1]["val_acc"]
            save_path = self.model_name + str(self.best_val_acc) + "best.p"
            pl_module.save(save_path)
            print("Saved checkpoint at epoch {} at \"{}\"".format((trainer.current_epoch + 1), save_path))
            
cb = SaveCallback("new_best_")

In [5]:
hparams = {
    "batch_size": 24,
    "learning_rate": 0.002,
    "weight_decay": 0.01,
    "lr_decay": 1
}

model = CRNNPLModule(hparams)
model.prepare_data()
datasetHandler.load(model, 'training')
datasetHandler.load(model, 'validation')

trainer = pl.Trainer(
    max_epochs=20,
    #logger= loggers.TensorBoardLogger(config.LOG_DIR, name=type(model)._name_),
    gpus=1 if torch.cuda.is_available() else None,
    callbacks=[cb]
)

trainer.fit(model)

Loading cached training data of dataset 0 from /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_48k/
Loading cached validation data of dataset 0 from /nfs/students/summer-term-2020/project-4/data/dataset1/dataset_48k/


GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]
/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: RuntimeWarning: You have defined a `val_dataloader()` and have defined a `validation_step()`, you may also want to define `validation_epoch_end()` for accumulating stats.
  warnings.warn(*args, **kwargs)
Set SLURM handle signals.

   | Name                          | Type                  | Params
--------------------------------------------------------------------
0  | model                         | CRNN                  | 255 K 
1  | model.spec                    | MelspectrogramStretch | 0     
2  | model.spec.spectrogram        | Spectrogram           | 0     
3  | model.spec.mel_scale          | MelScale              | 0     
4  | model.spec.stft               | Spectrogram           | 0     
5  | model.spec.random_stretch   

Val-Acc=0.00889152341434499


/nfs/students/summer-term-2020/project-4/yan/.conda/envs/cnn_project/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:23: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Val-Acc=0.8393598103141672
Train-Acc=0.807905138339921
new best val acc 0.8393598103141672
Saved model to "new_best_0.8393598103141672best.p"
Saved checkpoint at epoch 1 at "new_best_0.8393598103141672best.p"


...

Val-Acc=0.8707765263781861
Train-Acc=0.8660079051383399
new best val acc 0.8707765263781861
Saved model to "new_best_0.8707765263781861best.p"
Saved checkpoint at epoch 14 at "new_best_0.8707765263781861best.p"


Val-Acc=0.8369887374036752
Train-Acc=0.8598814229249012


Val-Acc=0.8761114404267931
Train-Acc=0.8588932806324111
new best val acc 0.8761114404267931
Saved model to "new_best_0.8761114404267931best.p"
Saved checkpoint at epoch 16 at "new_best_0.8761114404267931best.p"


Detected KeyboardInterrupt, attempting graceful shutdown...


1

In [6]:
# model.save("/nfs/students/summer-term-2020/project-4/SAVED_MODELS/SpectrogramCNN/vanilla.p")